Step 1: Load the Data

In [1]:
import pandas as pd

# Load game details and metadata
games = pd.read_csv('../data/games.csv')
metadata = pd.read_json('../data/games_metadata.json', lines=True)

# Preview the data
print("Games shape:", games.shape)
print("Metadata shape:", metadata.shape)
games.head()


Games shape: (50872, 13)
Metadata shape: (50872, 3)


,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [2]:
print(games.columns.tolist())


['app_id', 'title', 'date_release', 'win', 'mac', 'linux', 'rating', 'positive_ratio', 'user_reviews', 'price_final', 'price_original', 'discount', 'steam_deck']


Merge Datasets on app_id

In [3]:
# Merge base game info and metadata
merged_df = pd.merge(games, metadata, on='app_id')

# Now merge extra platform/price/date info from games.csv
columns_to_add = ['app_id', 'date_release', 'win', 'mac', 'linux', 'price_final', 'user_reviews']
merged_df = pd.merge(merged_df, games[columns_to_add], on='app_id', how='left')


Create combined_features for TF-IDF

In [4]:
# Fill NaN values
merged_df['tags'] = merged_df['tags'].fillna('').astype(str)
merged_df['description'] = merged_df['description'].fillna('').astype(str)

# -----------------------
# Clean Title Formatting
# -----------------------
import re

def clean_title(title):
    title = re.sub(r"[^\x00-\x7F]+", "", title)  # remove non-ASCII characters
    title = title.replace("™", "").replace("®", "").replace("â", "").replace("Â", "")
    return title.strip()

merged_df['title'] = merged_df['title'].astype(str).apply(clean_title)

# -----------------------
# Clean Genre Tags
# -----------------------
import ast

# Clean genre tags
def clean_genre(genre_string):
    try:
        genres = ast.literal_eval(genre_string)
        if isinstance(genres, list):
            genres = [g.strip("\"' ") for g in genres if isinstance(g, str)]
            allowed_genres = ['Action', 'Adventure', 'Horror', 'Puzzle', 'Strategy', 'RPG', 'Simulation']
            genres = [g for g in genres if g in allowed_genres]
            return genres
    except Exception:
        return []
    return []

# Apply cleaning
merged_df['tags'] = merged_df['tags'].apply(clean_genre)

# Join list into string
merged_df['tags'] = merged_df['tags'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '')


# Preview cleaned result
print(merged_df[['title', 'tags']].head())


                              title                       tags
0  Prince of Persia: Warrior Within    Action Adventure Puzzle
1           BRINK: Agents of Change                     Action
2      Monaco: What's Yours Is Mine  Strategy Action Adventure
3                Escape Dead Island    Adventure Action Horror
4            Dungeon of the ENDLESS     Strategy RPG Adventure


In [5]:
# ------------------------------------
# Convert genre list back to string
# ------------------------------------
merged_df['tags'] = merged_df['tags'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

# ------------------------------------
# Create combined_features column
# ------------------------------------
merged_df['combined_features'] = merged_df['tags'] + ' ' + merged_df['description']

# ------------------------------------
# Assign mood labels
# ------------------------------------
def assign_mood(row):
    description = row['description'].lower()
    tags = row['tags'].lower()
    if 'horror' in tags or 'zombies' in description:
        return 'horror'
    elif 'adventure' in tags:
        return 'fun'
    elif 'puzzle' in tags or 'logic' in description:
        return 'brainy'
    elif 'survival' in tags or 'intense' in description:
        return 'intense'
    elif 'relax' in description:
        return 'relaxing'
    else:
        return 'general'

merged_df['mood'] = merged_df.apply(assign_mood, axis=1)

# ------------------------------------
# TF-IDF Vectorization
# ------------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pickle
import os

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(merged_df['combined_features'])

print("TF-IDF matrix shape:", tfidf_matrix.shape)

# ------------------------------------
# Train KNN
# ------------------------------------
knn = NearestNeighbors(n_neighbors=16, metric='cosine')
knn.fit(tfidf_matrix)

# ------------------------------------
# Rename columns for UI compatibility
# ------------------------------------
merged_df.rename(columns={
    'price_final_x': 'price',
    'date_release_x': 'release_date',
    'win_x': 'win',
    'mac_x': 'mac',
    'linux_x': 'linux',
    'user_reviews_x': 'user_reviews'
}, inplace=True)

# ------------------------------------
# Save model and matrix
# ------------------------------------
os.makedirs('../models', exist_ok=True)

with open('../models/knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)

with open('../models/tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

# ------------------------------------
# Export cleaned dataset
# ------------------------------------
merged_df.to_csv("../data/cleaned_games.csv", index=False)

# Preview
merged_df[['title', 'mood', 'combined_features']].head()


TF-IDF matrix shape: (50872, 50286)


,title,mood,combined_features
0,Prince of Persia: Warrior Within,general,Enter the dark underworld of Prince of Persia...
1,BRINK: Agents of Change,general,
2,Monaco: What's Yours Is Mine,general,Monaco: What's Yours Is Mine is a single play...
3,Escape Dead Island,general,Escape Dead Island is a Survival-Mystery adve...
4,Dungeon of the ENDLESS,general,Dungeon of the Endless is a Rogue-Like Dungeo...


Phase 3: TF-IDF + KNN Model Training

Step 1: Import Libraries

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pickle


Step 2: Vectorize Combined Features with TF-IDF

In [7]:
# Vectorize combined_features
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(merged_df['combined_features'])

print("TF-IDF matrix shape:", tfidf_matrix.shape)


TF-IDF matrix shape: (50872, 50286)


 Step 3: Train KNN Model

In [8]:
# Train NearestNeighbors model
knn = NearestNeighbors(n_neighbors=6, metric='cosine')
knn.fit(tfidf_matrix)


NearestNeighbors(metric='cosine', n_neighbors=6)

Step 4: Save Your Model and Matrix as .pkl

In [9]:
import os

# Make sure 'models/' folder exists
os.makedirs('../models', exist_ok=True)


In [10]:
# Save model and matrix
with open('../models/knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)

with open('../models/tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)


In [11]:
# Export cleaned and combined dataset for the UI
merged_df.to_csv("../data/cleaned_games.csv", index=False)


Adding Mood Filtering


In [12]:
# Add a simple mood label manually (just a demo — real model can expand later)
def assign_mood(row):
    if 'horror' in row['tags'].lower() or 'zombies' in row['description'].lower():
        return 'horror'
    elif 'adventure' in row['tags'].lower():
        return 'fun'
    elif 'puzzle' in row['tags'].lower() or 'logic' in row['description'].lower():
        return 'brainy'
    elif 'survival' in row['tags'].lower() or 'intense' in row['description'].lower():
        return 'intense'
    elif 'relax' in row['description'].lower():
        return 'relaxing'
    else:
        return 'general'

# Apply mood labeling
merged_df['mood'] = merged_df.apply(assign_mood, axis=1)

# Preview
merged_df[['title', 'tags', 'mood']].head()


,title,tags,mood
0,Prince of Persia: Warrior Within,,general
1,BRINK: Agents of Change,,general
2,Monaco: What's Yours Is Mine,,general
3,Escape Dead Island,,general
4,Dungeon of the ENDLESS,,general


In [13]:
# remove duplicates and brackets from the tags column
def clean_tags(tags):
    # Remove brackets and duplicates
    tags = pd.Series(tags).str.replace(r'\[|\]', '', regex=True).iloc[0]
    tags = ' '.join(sorted(set(tags.split())))
    return tags
merged_df['tags'] = merged_df['tags'].apply(clean_tags)
# Save cleaned dataset
merged_df.to_csv("../data/cleaned_games.csv", index=False)

In [14]:
# Create a function that removes the little trademark symbol from the titles
def clean_title(title):
    return title.replace('™', '').strip()
# Apply the function to the title column
merged_df['title'] = merged_df['title'].apply(clean_title)
# Save the cleaned DataFrame to a new CSV file
merged_df.to_csv("../data/cleaned_games.csv", index=False)


In [15]:
# Create the function that removes the registered trademark symbol from the titles
def clean_title(title):
    return title.replace('®', '').strip()
# Apply the function to the title column
merged_df['title'] = merged_df['title'].apply(clean_title)
# Save the cleaned DataFrame to a new CSV file
merged_df.to_csv("../data/cleaned_games.csv", index=False)

In [16]:
merged_df.rename(columns={
    'price_final_x': 'price',
    'date_release_x': 'release_date',
    'win_x': 'win',
    'mac_x': 'mac',
    'linux_x': 'linux',
    'user_reviews_x': 'user_reviews'
}, inplace=True)


In [17]:
merged_df.rename(columns={
    'price_final': 'price',
    'date_release': 'release_date',
    'win': 'win',
    'mac': 'mac',
    'linux': 'linux',
    'user_reviews': 'user_reviews',
    'playtime_forever': 'hours'  # use this if your hours column is named something else
}, inplace=True)


In [18]:
print(merged_df.columns.tolist())


['app_id', 'title', 'release_date', 'win', 'mac', 'linux', 'rating', 'positive_ratio', 'user_reviews', 'price', 'price_original', 'discount', 'steam_deck', 'description', 'tags', 'date_release_y', 'win_y', 'mac_y', 'linux_y', 'price_final_y', 'user_reviews_y', 'combined_features', 'mood']


Save updated CSV again

In [19]:
merged_df.to_csv("../data/cleaned_games.csv", index=False)


In [20]:
merged_df.rename(columns={
    'price_final_x': 'price',
    'date_release_x': 'release_date',
    'win_x': 'win',
    'mac_x': 'mac',
    'linux_x': 'linux',
    'user_reviews_x': 'user_reviews'
}, inplace=True)
